#### Stage two: Big Data Project

Team Members: <br>
Sumanth V Rao - 01FB16ECS402 <br>
Sumedh Pb - 01FB16ECS403 <br>
Tanmaya Udupa - 01FB16ECS416<br>
Siddhart Itagi - 01FB16ECS382<br>

In [1]:
import random
import pandas as pd
from pyspark.sql.functions import count,col,when,sum
spark.conf.set("spark.sql.crossJoin.enabled", "true")

In [2]:
df = spark.read.options(header="true",
                               inferSchema="true",
                               nullValue="-",
                               mode="failfast").csv("file:///home/sumanthvrao/bigdata/2016_17.csv")

In [3]:
cluster_batsman = spark.read.options(header="true",
                               inferSchema="true",
                               nullValue="-",
                               mode="failfast").csv("file:///home/sumanthvrao/bigdata/cluster_batsman.csv")

In [4]:
cluster_bowler = spark.read.options(header="true",
                               inferSchema="true",
                               nullValue="-",
                               mode="failfast").csv("file:///home/sumanthvrao/bigdata/cluster_bowler.csv")

In [5]:
df.printSchema()

root
 |-- date: integer (nullable = true)
 |-- batsman: string (nullable = true)
 |-- bowler: string (nullable = true)
 |-- runs: integer (nullable = true)
 |-- wickets: string (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- winning team: string (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- type_of_wicket: string (nullable = true)
 |-- person_responsible: string (nullable = true)
 |-- player_out: string (nullable = true)



In [6]:
#27,945 rows from 2016,2017 data
df.count()

27945

In [7]:
df.toPandas()[:5]

   date    batsman     ...     person_responsible  player_out
0  2017  DA Warner     ...                      0           0
1  2017  DA Warner     ...                      0           0
2  2017  DA Warner     ...                      0           0
3  2017  DA Warner     ...                      0           0
4  2017  DA Warner     ...                      0           0

[5 rows x 12 columns]

In [8]:
#get the wickets probability for each batsmen-bowler combination in the dataset. This should be run only once, prehand

wickets=[]
def initialize_wickets():
   
    all_batsman = cluster_batsman.select("Player")
    all_bowler = cluster_bowler.select("Player")
    #caching the RDD's
    
    cross_product = all_batsman.join(all_bowler).toDF("batsman","bowler").collect()
    #print(len(cross_product))
    l = len(cross_product)
    for i in range (0,l):
        str1 = cross_product[i][0]
        str2 = cross_product[i][1]
        prob = g_g_wickets(str1,str2)
        wickets.append(prob)
    
initialize_wickets()


Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/tmp/kernel-PySpark-ffa328af-de1f-41df-8426-93ba7ce99336/pyspark_runner.py", line 194, in <module>
    eval(compiled_code)
  File "<string>", line 12, in <module>
  File "<string>", line 10, in initialize_wickets
NameError: name 'g_g_wickets' is not defined

StackTrace: org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:163)
org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:163)
scala.Option.foreach(Option.scala:257)
org.apache.toree.interpreter.broker.BrokerState.markFailure(BrokerState.scala:162)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
java.lang.reflect.Method.invoke(Method.java:498)
py4j.reflection.MethodIn

In [9]:
def g_g_wickets(str1,str2):
    batsman = cluster_batsman.where(cluster_batsman.Player==str1)
    bats_c_no = batsman.select("prediction").collect()[0][0]
    
    bowler = cluster_bowler.where(cluster_bowler.Player==str2)
    bowl_c_no = bowler.select("prediction").collect()[0][0]
    
    all_batsman = cluster_batsman.where(cluster_batsman.prediction==bats_c_no)
    list_batsman= all_batsman.select("Player")
    
    all_bowler = cluster_bowler.where(cluster_bowler.prediction==bowl_c_no)
    list_bowler=all_bowler.select("Player")
    
    #caching the RDD's
    
    cross_product = list_batsman.join(list_bowler).toDF("batsman","bowler").cache()
    df_pairs = df.join(cross_product,["batsman","bowler"],"inner").cache()
    
    denom = df_pairs.count()
    inter = df_pairs.groupBy("batsman","bowler") #caching this RDD
    wickets = inter.agg(count(when(col("wickets")=="out",True)))
    no_wickets= wickets.groupBy().sum().collect()[0][0]
    
    prob = no_wickets/denom
    print("batsman: "+str1+" bowler: "+str2+" prob: "+str(prob))
    return prob

In [10]:
#Using Clustering method to determine the runs scored (for rare combination)


def group_group(str1,str2):
    batsman = cluster_batsman.where(cluster_batsman.Player==str1)
    bats_c_no = batsman.select("prediction").collect()[0][0]
    
    bowler = cluster_bowler.where(cluster_bowler.Player==str2)
    bowl_c_no = bowler.select("prediction").collect()[0][0]
    
    all_batsman = cluster_batsman.where(cluster_batsman.prediction==bats_c_no)
    list_batsman= all_batsman.select("Player")
    
    all_bowler = cluster_bowler.where(cluster_bowler.prediction==bowl_c_no)
    list_bowler=all_bowler.select("Player")
    
    #caching the RDD's
    
    cross_product = list_batsman.join(list_bowler).toDF("batsman","bowler").cache()
    df_pairs = df.join(cross_product,["batsman","bowler"],"inner").cache()
    
    
    denom = df_pairs.count()
    inter = df_pairs.groupBy("batsman","bowler") #caching this RDD
    
    
    zeros = inter.agg(count(when(col("runs")==0,True)))
    no_zeros= zeros.groupBy().sum().collect()[0][0]
    
    
    ones =  inter.agg(count(when(col("runs")==1,True)))
    no_ones= ones.groupBy().sum().collect()[0][0]
    
    twos =  inter.agg(count(when(col("runs")==2,True)))
    no_twos= twos.groupBy().sum().collect()[0][0]
    
    threes =  inter.agg(count(when(col("runs")==3,True)))
    no_threes= threes.groupBy().sum().collect()[0][0]
    
    fours=  inter.agg(count(when(col("runs")==4,True)))
    no_fours= fours.groupBy().sum().collect()[0][0]
    
    sixes =  inter.agg(count(when(col("runs")==6,True)))
    no_sixes= sixes.groupBy().sum().collect()[0][0]
    g_0 = no_zeros/denom
    g_1 = no_ones/denom
    g_2 = no_twos/denom
    g_3 = no_threes/denom
    g_4 = no_fours/denom
    g_6 = no_sixes/denom

    runs=[g_0,g_1,g_2,g_3,g_4,g_6]
    c_runs = [g_0]
    for i in range(1,len(runs)):
        c_runs.append(runs[i]+c_runs[i-1])
        
    rand = random.random()
    
    if(c_runs[0] <=rand):
        if(c_runs[1]<=rand):
            if(c_runs[2]<=rand):
                if(c_runs[3]<=rand):
                    if(c_runs[4]<=rand):
                        index=6
                    else:
                        index=4
                else:
                    index=3
            else:
                index=2
        else:
            index=1
    else:
        index=0

    #print(no_zeros,no_ones,no_twos,no_threes,no_fours,no_sixes,denom)
    #print(runs)
    #print(c_runs)
    #print(rand)
    print("Insufficient data. Clustering based runs: ",index)
    return index

In [11]:
#Using the naive method to determine the runs scored

def player_player(str1,str2):
    #global comment
    b1 = df.where((df.batsman==str1) & (df.bowler==str2)).select("batsman","bowler","runs","wickets")
    
    rand = random.random()
    if(b1.count()< 5):
        score = group_group(str1,str2)
        return score
    else:
        runs_0 = b1.where(b1.runs==0)
        count_0= runs_0.count()
            
        runs_1 = b1.where(b1.runs==1)
        count_1= runs_1.count()
        
        runs_2 = b1.where(b1.runs==2)
        count_2= runs_2.count()
            
        runs_3 = b1.where(b1.runs==3)
        count_3= runs_3.count()
            
        runs_4 = b1.where(b1.runs==4)
        count_4= runs_4.count()
        
        runs_6 = b1.where(b1.runs==6)
        count_6= runs_6.count()
            
        count = b1.count()
        
        p_0 = count_0/count
        p_1 = count_1/count
        p_2 = count_2/count
        p_3 = count_3/count
        p_4 = count_4/count
        p_6 = count_6/count
          
        runs=[p_0,p_1,p_2,p_3,p_4,p_6]
        c_runs = [p_0]
        for i in range(1,len(runs)):
            c_runs.append(runs[i]+c_runs[i-1])
        
        if(c_runs[0] <=rand):
            if(c_runs[1]<=rand):
                if(c_runs[2]<=rand):
                    if(c_runs[3]<=rand):
                        if(c_runs[4]<=rand):
                            index=6
                        else:
                            index=4
                    else:
                        index=3
                else:
                    index=2
            else:
                index=1
        else:
            index=0
                
        #b1.show()
        #print(runs)
        #print(c_runs)
        #print(rand)
        print("It's a :"+str(index))
        #comment+="It's a :"+str(index)+"\n"
        return index
#score = player_player("KA Pollard","MM Sharma")
#print(score)


In [12]:
#DRIVER functions
#Two batsmen's not out probabilities (who are currently batting)
#For a complete 
no_x=1 #striker
no_y=1 #non-striker

balls_left=6

def driver(str1,str2,str3,chance):
    global no_x
    global score_1
    global comment
    global score_2
    global wicket_1
    global wicket_2
    global balls_left
    while(balls_left>0):
        balls_left=balls_left-1
        hshs = answer.where((answer.batsman==str1) & (answer.bowler==str2)).collect()
        if(len(hshs)==0):
            temp=1.0
        else:
            temp=(1-answer.where((answer.batsman==str1) & (answer.bowler==str2)).select("ratio").collect()[0][0])*no_x
        #print(temp)
        if(temp > 0.5):
            #call player_player and compute runs
            runs = player_player(str1,str2)
            no_x=temp
            if(chance==1):
                score_1+=runs
            else:
                score_2+=runs
        else:
            #Batsman got out
            no_x=temp
            if(chance==1):
                wicket_1+=1
            else:
                wicket_2+=1
            return(balls_left)
        
    balls_left=6
    if(chance==1):
        print("End of over. Score : ",score_1,"/",wicket_1)
        comment+="End of over. Score :"+str(score_1)+"/"+str(wicket_1)+"\n"
    else:
        print("End of over. Score : ",score_2,"/",wicket_2)
        comment+="End of over. Score :"+str(score_2)+"/"+str(wicket_2)+"\n"
    return(balls_left)
    

In [13]:
#SET INPUTS

#batsman_1=["R Sathish","SS Tiwary"]
#batsman_2=["KA Pollard","Shoaib Malik"]
#bowler_2=["CR Brathwaite"]
#bowler_1=["MM Sharma"]

batsman_1=[
"RG Sharma",
"Ishan Kishan" 
,"SA Yadav"
,"HH Pandya"
,"KH Pandya"
,"KA Pollard"
"MJ McClenaghan"
,"Mustafizur Rahman"
,"JJ Bumrah"]

bowler_2=[
"SR Watson",
"Imran Tahir",
"SR Watson",
"Imran Tahir",
"DJ Bravo" ,
"Harbhajan Singh",
"DL Chahar",
"Harbhajan Singh",
"RA Jadeja",
"DL Chahar",
"DJ Bravo",
"Harbhajan Singh",
"DL Chahar",
"Imran Tahir",
"DJ Bravo",
"Imran Tahir",
"SR Watson" ,
"Harbhajan Singh",
"DJ Bravo",
"SR Watson"
]

batsman_2=[
"SR Watson",
"AT Rayudu",
"SK Raina",
"KM Jadhav",
"MS Dhoni",
"RA Jadeja",
"DJ Bravo",
"DL Chahar",
"Harbhajan Singh",
"Imran Tahir"]

bowler_1=[
"MJ McClenaghan",
"JJ Bumrah",
"MJ McClenaghan",
"JJ Bumrah",
"KH Pandya",
"Mustafizur Rahman",
"HH Pandya",
"Mustafizur Rahman",
"HH Pandya",
"KH Pandya",
"Mustafizur Rahman",
"HH Pandya",
"KH Pandya",
"Mustafizur Rahman",  
"JJ Bumrah",
"KH Pandya",
"HH Pandya",
"MJ McClenaghan",
"JJ Bumrah",
"MJ McClenaghan"  
]

score_1=0
score_2=0
wicket_1=0
wicket_2=0

i=0 #i is index of batsman
k=1 #next_striker
j=0 #bowler index

winner=''
team_1=''
team_2=''


In [14]:
def main_caller():
    print("Team 1 batting, Team 2 bowling")
    global comment
    
    comment+="Team 1 batting, Team 2 bowling\n"
    global batsman_1
    global batsman_2
    global bowler_2
    global bowler_1
    
    global score_1
    global score_2
    global wicket_1
    global wicket_2
    
    global i
    global k
    global j
    
    global no_x
    global no_y
    global balls_left
    
    global winner
   
    s_batsman = batsman_1[0]
    ns_batsman = batsman_1[1]
    while(k<len(batsman_1) and j<len(bowler_2)):
        bowler=bowler_2[j]
        balls_left = driver(s_batsman,bowler,ns_batsman,1)

        if(balls_left==6):
            j+=1
            (s_batsman,ns_batsman)=(ns_batsman,s_batsman)
            (no_x,no_y)=(no_y,no_x) #swapped striker and non striker
            print(str(score_1)+"/"+str(wicket_1))
            comment+=str(score_1)+"/"+str(wicket_1)+"\n"

        else:
            k+=1
            no_x=1
            print("Wicket! ,"+str(s_batsman)+" got out")
            print(str(score_1)+"/"+str(wicket_1))
            
            comment+="Wicket! ,"+str(s_batsman)+" got out\n"
            comment+=str(score_1)+"/"+str(wicket_1)+"\n"
            if(k<len(batsman_1)):
                s_batsman=batsman_1[k]

    print("Innings 1 ended")
    comment+="Innings 1 ended\n"
    
    #Initializing & Resetting 
    no_x=1
    no_y=1
    i=0 #i is index of batsman
    k=1 #next_striker
    j=0 #bowler index
    balls_left=6

    print("-----------------------------------")
    comment+="------------------------\n"
    print("Team 2 batting, Team 1 bowling")
    comment+="Team 2 batting, Team 1 bowling\n"

    s_batsman = batsman_2[0]
    ns_batsman = batsman_2[1]
    while(k<len(batsman_2) and j<len(bowler_1)):
        bowler=bowler_1[j]
        balls_left = driver(s_batsman,bowler,ns_batsman,2)

        if(balls_left==6):
            j+=1
            (s_batsman,ns_batsman)=(ns_batsman,s_batsman)
            (no_x,no_y)=(no_y,no_x) #swapped striker and non striker
            print(str(score_2)+"/"+str(wicket_2))
            comment+=str(score_2)+"/"+str(wicket_2)+"\n"

        else:
            k+=1
            no_x=1
            print("Wicket! ,"+str(s_batsman)+" got out")
            print(str(score_2)+"/"+str(wicket_2))
            
            comment+="Wicket! ,"+str(s_batsman)+" got out\n"
            comment+=str(score_2)+"/"+str(wicket_2)+"\n"
            
            if(k<len(batsman_2)):
                s_batsman=batsman_2[k]
    print("Match ended")
    comment+="Match ended\n"
    
    winner = team_1 if score_1>score_2 else team_2
    result="Winning team is %s"%(winner)
    tkinter.messagebox.showinfo(message=comment)
    tkinter.messagebox.showinfo(message=result)
    
    #print("The winner is _________ with score",winner)

In [15]:
from tkinter import *
import tkinter as tk
import tkinter.messagebox
import tkinter.simpledialog
import sys

mw = tk.Tk()

mw.option_add("*Button.Background", "black")
mw.option_add("*Button.Foreground", "green")
mw.option_add('*Dialog.msg.font', 'Helvetica 15')

mw.title('Cricket Simulator')
mw.geometry("900x700") 

back = tk.Frame(master=mw,bg='black')
back.pack_propagate(0)
back.pack(fill=tk.BOTH, expand=1) 

info = tk.Label(master=back, text='Ball by ball simulations of any IPL 2018 match of your choice', bg='green',font=(None,20),fg='black')
info.pack()

def retrieve_input():
    global team_1
    global team_2
    team_1=textBox1.get("1.0","end-1c")
    team_2 = textBox2.get("1.0","end-1c")
    output="Team 1 is %s\n"%(team_1)
    output+="Team 2 is %s"%(team_2)
    tkinter.messagebox.showinfo('Teams',output)
    
Label(back, text="First Team",bg='green',font=(None,20)).pack(side='top')
textBox1=Text(back, height=2, width=80)
textBox1.pack(side='top')

Label(back, text="Second Team",bg='green',font=(None,20)).pack(side='top')
textBox2=Text(back, height=2, width=80)
textBox2.pack(side='top')

buttonCommit=Button(back, height=1, width=10, text="Commit", 
                    command=lambda: retrieve_input())
buttonCommit.pack()

comment =''
boolean_val = 0
go = tk.Button(master=back, text='Start Simulation', font=(None,50), command=main_caller)
go.pack()


close = tk.Button(master=back, text='Quit',font=(None,30), command=mw.destroy)
close.pack(side='bottom')

mw.mainloop()